In [14]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
!pip install peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.1 MB/s eta 0:00:0000:0100:01


In [15]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load the CSV file into a DataFrame
file_path = '/kaggle/input/megathon-data/Final_Dataset.csv'  # Replace with your CSV file path
data = pd.read_csv(file_path)

print(data.columns)

Index(['User Input', ' Polarity', ' Extracted Concern', ' Category',
       ' Intensity'],
      dtype='object')


In [18]:
# Extract concerns and categories from the DataFrame
concerns = data[' Extracted Concern'].tolist()  # Adjust column name as necessary
categories = data[' Category'].tolist()  # Adjust column name as necessary

# Specify model path
model_name = '/kaggle/input/mistral/pytorch/7b-v0.1-hf/1'

# Load tokenizer and model using specified model path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Define a function to predict category based on concern
def predict_category(concern):
    output = pipe(concern, max_length=50, num_return_sequences=1)
    return output[0]['generated_text'].strip()

# Predict categories for each concern in the dataset
predictions = [predict_category(concern) for concern in concerns]

# Print the predictions along with their corresponding concerns and actual categories
for concern, prediction, actual in zip(concerns, predictions, categories):
    print(f"Concern: {concern}\nPredicted Category: {prediction}\nActual Category: {actual}\n")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end gener

Concern:  feeling much better
Predicted Category: feeling much better today.

i’m still not 100% but i’m feeling much better.

i’m still not 100% but i’m feeling much better.

i
Actual Category:  General Wellbeing

Concern:  feeling very anxious
Predicted Category: feeling very anxious about the future of the world.

I’m not sure if it’s the fact that I’m getting older and I’m starting to see the world in a different way, or if it’s
Actual Category:  Anxiety

Concern:  feeling very low
Predicted Category: feeling very low

I have been feeling very low for the past few days. I have been feeling very low for the past few days. I have been feeling very low for the past few days. I have been feeling very low for
Actual Category:  Depression

Concern:  happy and excited
Predicted Category: happy and excited to announce that i’m now a part of the team at the new york times’s cooking section! i’ll be contributing recipes and stories to the site, and i’m so excited to be a part
Actual Category